In [4]:
import pdfminer

In [2]:
import re
import pandas as pd
import numpy as np
import pickle

from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

from urllib2 import urlopen, Request
from StringIO import StringIO as sIO

In [11]:
import requests
from bs4 import BeautifulSoup, UnicodeDammit

In [12]:
def url_to_pdf(url):
    open = urlopen(Request(url)).read()
    return sIO(open)

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    #fp = file(path, 'rb')
    fp = url_to_pdf(path)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    pagenos=set()
    for i, page in enumerate(PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True)):
        #if i > 0:
        interpreter.process_page(page)
    fp.close()
    device.close()
    str = retstr.getvalue()
    retstr.close()
    return str

In [13]:
def get_pdf_text(start, n_days):
    delta = pd.Timedelta(1, 'd')
    pdfs = {}
    for i in xrange(n_days):
        base = (start + i*delta).strftime("%Y%m%d")
        for j in xrange(100):
            link = 'http://appsrv.achd.net/reports/rwservlet?food_rep_insp&P_ENCOUNTER=%s%04d' % (base,j+1)
            z = requests.get(link)
            if z.headers['Content-Type'] == 'application/pdf':
                pdfs['%s%04d' % (base, j+1)] = {'link': link, 'text': convert_pdf_to_txt(link)}
            else:
                break
        print '%s max: %d' % (base, j)
    return pdfs

def save_pdf_text(pdfs, fname):
    with open(fname, 'wb') as handle:
        pickle.dump(pdfs, handle)

def merge_two_dicts(x, y):
    '''Given two dicts, merge them into a new dict as a shallow copy.'''
    z = x.copy()
    z.update(y)
    return z

In [41]:
i = 2

In [222]:
#with open('data/pitt/pitt_20110101_to_20110601.pkl', 'rb') as f:
with open('data/pitt/pitt_20110602_to_20111229.pkl', 'rb') as f:
    X = pickle.load(f)
test = X[X.keys()[i]]['text']
#test = convert_pdf_to_txt('http://appsrv.achd.net/reports/rwservlet?food_rep_insp&P_ENCOUNTER=201409080018')
#test

In [47]:
i = 2
test = X[X.keys()[i]]['text']

348

In [301]:
l = []
names = [0,1,2,3,4,5,6,7,8,9,10,13,32,35,36,44,46]
names = [8,9,15,17,21,24]
names = [656,2464,2813,4108,4346,655,2855,2371,2578,3626,56,158,312,991]
names = [1059,4488]
names = [4496]
names = [468, 1043, 4785, 5197, 6161]
names = [32,331,852,1329]
names = [99,723,839,686,829,1185,2350,5829,4269,22,271,791,601,985,1177]
names = [468,1043,5197]
names = [4785,6161]
names = [4496]
for i in names:
    test = X[X.keys()[i]]['text']
    t = test.split('\nPage 1 of 2\n\n')
    l.append(t[0].split('\n')[:50])
    
pd.DataFrame(zip(*l), columns=names)

,4496
0,Allegheny County Health Department
1,Food Safety Program
2,"3901 Penn Ave,"
3,"Pittsburgh, PA 15224"
4,Phone: 412-578-8044 Fax: 412-578-8190
5,
6,Report
7,
8,Food Safety Assessment
9,


In [302]:
def is_number(s):
    try:
        float(s)
        return True
    except:
        return False

def get_restaurant_details(pdf):
    t = pdf.split('\n')
    f = t.index('Food Safety Assessment Categories:')
    n = t.index('Client Name:')
    z = t.index('Zip:')
    p = t.index('Priority Code:')
    
    if t[f+2] == 'Client Name:':
        f += 1
    
    i1 = t.index('1')
    i2 = t.index('2')
    if 'H' in t:
        i_priority = min([i1,i2,t.index('H')])
    else:
        i_priority = min([i1,i2])
    
    d =    {'client_id': t[f+2],
            'address': t[f+3],
            'city': t[f+4],
            'municipality': t[f+5],
            'category': t[f+6],
            'reinspection': t[f+7],
            'name': t[n + 2],
            'zip': t[z - 2],
            'inspector': t[z + 1],
            'date': t[p + 2],
            'purpose': t[p + 3],
            'priority': t[i_priority]}
    
    # Check for several known exceptions in field order:
    if d['name'] == 'State:':
        d['name'] = t[t.index('Inspector:') + 2]
        
    if is_number(d['inspector']):
        d['inspector'] = t[z + 2]
        d['zip'] = t[z + 1]
        
    if d['zip'] == 'Priority Code:':
        if t[p+8] == '':
            d['zip'] = t[z+3]
            d['date'] = t[t.index('Inspection Date:') + 1]
        else:
            d['zip'] = t[p + 8]
            d['date'] = t[p + 5]
        d['purpose'] = t[p + 6]
        
    if len(d['date']) == 5 and is_number(d['date']):
        i = t.index('Inspection Date:')
        d['date'] = t[i - 3]
        d['purpose'] = t[i - 2]
    elif len(d['date']) < 6:
        d['date'] = t[t.index('Inspection Date:') + 1]
    elif d['date'] == 'Purpose:':
        i = t.index('Inspection Date:')
        d['zip'] = t[i - 2]
        d['date'] = t[i - 5]
        d['purpose'] = t[i - 4]
    
    if t[f+1] == 'Client Name:':
        d['name'] = t[t.index('Inspector:')+4]
    
    if d['city'] == 'Zip:':
        r = map(lambda x: re.search('Re- Inspection Date:',x) is not None, t).index(True)
        if is_number(d['municipality']):
            f += 1
        d['city'] = t[f+5]
        d['municipality'] = t[f+7]
        d['category'] = t[f+8]
        d['reinspection'] = t[f+9]
        d['inspector'] = t[f+6]
        if is_number(t[r+2]):
            d['name'] = t[r+4]
            d['zip'] = t[r+2]
        else:
            d['name'] = t[r+2]
            if t[r+4] == 'Inspection Date:':
                d['zip'] = t[z+1]
            else:
                d['zip'] = t[r+4]
    elif d['city'] == 'State:':
        d['city'] = t[f+6]
        d['municipality'] = t[f+8]
        d['category'] = t[f+9]
        d['reinspection'] = t[f+10]
    
    if d['name'] == 'Client ID:':
        i = t.index('Inspection Date:')
        if t[i - 1] == '':
            d['name'] = t[i - 2]
        else:
            d['name'] = t[i - 1]
       
    if is_number(d['purpose']):
        if d['date'] == 'Purpose:':
            d['date'] = t[z+4]
            d['purpose'] = t[z+5]
        else:
            d['purpose'] = t[t.index('Purpose:') + 1]
    elif d['purpose'] == '':
        d['purpose'] = t[t.index('Purpose:') + 1]
            
    return d
        
    
def get_violation_details(pdf):
    #t = pdf.split('\nPage 1 of 2\n\n')
    v =  pdf.split('Inspection Details')[1]\
            .replace('\n\nViolation:\nComments:\nFood Code Section(s):\nCorrective Action:\n\n','\n\n')\
            .replace('Corrective Action:\n\n','\n\n')\
            .replace('\nOther Assesment observations and comments:\n\n','\n\n')\
            .replace('Non Critical Violations:','Non Critical Violations:\n')\
            .replace('\n3','\n\n3')\
            .replace('\n 3','\n\n3')\
            .replace('\n3\n','3\n')\
            .replace('\n\n\n','\n\n')\
            .replace('\n\n\n','\n\n')\
            .split('\n\n')
            
    critical = [int(x) for x in v if (is_number(x) and len(x) < 4)]
    noncritical = []
    
    if 'Non Critical Violations:' in v:
        ind_nc = v.index('Non Critical Violations:')+1
        #if 'Client #' in v:
        #    ind_stop = v.index('Client #')
        #else:
        ind_stop = len(v)
        pattern = re.compile('\b?3\d\d\b?')
        temp = [pattern.findall(re.sub('\d{12}','',x)) for x in v[ind_nc:ind_stop] if re.search('3901 Penn Ave', x) is None]
        
        pattern = re.compile('3\d\d')
        noncritical = [int(x[0]) for x in temp if len(x) > 0]
    return {'critical':critical, 'noncritical':noncritical}
    

In [303]:
# get restaurant details
l = {}
for i in xrange(len(X)):
    test = X[X.keys()[i]]['text']
    l[i] = get_restaurant_details(test)
    
df = pd.DataFrame(l).T
df.to_csv('data/pitt/test_rest_info2.csv')

In [304]:
# get violation details
l = {}
for i in xrange(len(X)):
#for i in xrange(10):
    test = X[X.keys()[i]]['text']
    try:
        l[i] = merge_two_dicts(get_restaurant_details(test), get_violation_details(test))
        l[i]['id'] = X.keys()[i]
    except Exception as e:
        print i, X.keys()[i], e
        break
                           
pd.DataFrame(l).T.to_csv('data/pitt/test_violation_info2.csv')

In [214]:
X['201103310048']['text']

"Allegheny County Health Department\nFood Safety Program\n3901 Penn Ave, \nPittsburgh, PA 15224\nPhone: 412-578-8044 Fax: 412-578-8190\n\nReport\n\nFood Safety Assessment \n\nClient ID:\nAddress:\nCity:\nMunicipality:\nCategory Code:\nRe- Inspection:\nFood Safety Assessment Categories:\n\n9426\n2240 Main St \nState:\nPA\nPittsburgh\nInspector:\nPenn Hills\n410-Community Service Facility/Exempt\nNo - Inspector\n\nRe- Inspection Date:\n\nClient Name:\n\nPenn Hills VFD No 6\n\n15235\n\nZip:\nAmmon , Cathy\n\nInspection Date:\nPurpose:\nPermit Exp. Date: 01/31/2015\nPriority Code:\n\n03/29/2011\nInitial\n\n2\n\nAssessment Status\n\nNO\n\nNA\n\nV\n\nx\n\nx\nx\nx\n\nx\n\nx\n\nx\n\nD\n\nS\n\nx\n\nx\nx\n\nx\nx\nx\nx\nx\nx\nx\nx\n\nx\nx\nx\nx\n\n0\n\nD\n\n15\n\n4\n\n2\n\nAssessment Status\n\nS\n\nNO\n\nNA\n\n1\n\nV\n\n1\n2\n3\n4\n5\n6\n7\n8\n9\n10\n11\n12\n13\n14\n15\n16\n17\n18\n19\n20\n21\n22\n23\n24\n25\n\nFood Source/Condition\nCooking Temperatures\nConsumer Advisory\nReheating Temperatures

In [49]:
l[0].index('Client Name:')

31

In [127]:
t = X['201101070019']['text'].split('\nPage 1 of 2\n\n')

In [45]:
t[0].split('\n')

['Allegheny County Health Department',
 'Food Safety Program',
 '3901 Penn Ave, ',
 'Pittsburgh, PA 15224',
 'Phone: 412-578-8044 Fax: 412-578-8190',
 '',
 'Report',
 '',
 'Food Safety Assessment ',
 '',
 'Client ID:',
 'Address:',
 'City:',
 'Municipality:',
 'Category Code:',
 'Re- Inspection:',
 'Food Safety Assessment Categories:',
 '',
 '56360',
 '3224 Brighton Rd ',
 'Pittsburgh',
 'Pittsburgh-127',
 '201-Restaurant with Liquor',
 'No - Inspector',
 '',
 'Client Name:',
 '',
 "O'Brien's Bar",
 '',
 'State:',
 'PA',
 'Inspector:',
 '',
 '15212',
 '',
 'Zip:',
 'Shafer , Jessica A.',
 '',
 'Re- Inspection Date:',
 '',
 'Inspection Date:',
 'Purpose:',
 'Permit Exp. Date: 04/30/2016',
 'Priority Code:',
 '',
 '03/03/2011',
 'Initial',
 '',
 '1',
 '',
 'Assessment Status',
 '',
 'D',
 '',
 'S',
 '',
 'x',
 '',
 'x',
 'x',
 '',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 'x',
 '',
 'x',
 'x',
 '',
 'x',
 '',
 'NO',
 '',
 'NA',
 '',
 'V',
 '',
 'x',
 'x',
 'x',
 'x',
 'x',
 '',
 'x',
 '

In [46]:
# client id, address, city, municipality, category code, re-inspection
# name, zip, inspector, insepction date, purpose
[t[0].split('\n')[x] for x in [18,19,20,21,22,23,29,33,35,38,45,46]]

['56360',
 '3224 Brighton Rd ',
 'Pittsburgh',
 'Pittsburgh-127',
 '201-Restaurant with Liquor',
 'No - Inspector',
 'State:',
 '15212',
 'Zip:',
 'Re- Inspection Date:',
 '03/03/2011',
 'Initial']

In [145]:
t = X['201101070019']['text'].split('\nPage 1 of 2\n\n')
v = t[1].split('Inspection Details')[1]\
        .replace('\n\nViolation:\nComments:\nFood Code Section(s):\nCorrective Action:\n\n','\n\n')\
        .replace('Corrective Action:\n\n','\n\n')\
        .replace('\nOther Assesment observations and comments:\n\n','\n\n')\
        .replace('Non Critical Violations:','Non Critical Violations:\n')\
        .replace('\n3','\n\n3')\
        .replace('\n 3','\n\n3')\
        .replace('\n3\n','3\n')\
        .replace('\n\n\n','\n\n')\
        .replace('\n\n\n','\n\n')\
        .split('\n\n')
v

['',
 '3',
 'Consumer Advisory',
 'No advisory on menu with made to order steaks, eggs. Provide.',
 'Inform consumers or DISCLOSE that raw or undercooked animal products are offered for service.\nProvide written notification or REMINDER of the health risk of eating raw or undercooked animal-\nderived products.',
 '6',
 'Hot Holding Temperatures',
 'Tamales on steam table at 120F and 84F.  Moved and re-layered; temp no 160F.',
 '303\nMaintain food at or above 135\xc2\xb0 F',
 '10',
 'Probe-Type Thermometers',
 'No batteries in thermometer.  Provide.',
 '304\nProvide probe-type thermometers with 0\xc2\xb0 F to 220\xc2\xb0 F range\nUse thermometer to monitor food temperatures',
 'Corrective Action:',
 '21',
 'Certified Food Protection Manager',
 'Certified manager not present.',
 '336\nProvide Certified Food Protection Manager to premises during all hours of operation\nCertified Food Protection Manager Course schedule left with person in charge',
 'Non Critical Comments',
 '(303) Unlabele

In [142]:
t = X['201103080007']['text'].split('\nPage 1 of 2\n\n')
v = t[1].split('Inspection Details')[1]\
        .replace('\n\nViolation:\nComments:\nFood Code Section(s):\nCorrective Action:\n\n','\n\n')\
        .replace('\nOther Assesment observations and comments:\n\n','\n\n')\
        .replace('Non Critical Violations:','Non Critical Violations:\n')\
        .replace('\n3','\n\n3')\
        .replace('\n 3','\n\n3')\
        .replace('\n3\n','3\n')\
        .replace('\n\n\n','\n\n')\
        .replace('\n\n\n','\n\n\n\n')\
        .split('\n\n')
v

['',
 '14',
 'Cleaning and Sanitization',
 '* Wiping tables with plain water.',
 '312\nClean and sanitize food-contact surfaces of equipment and utensils after each use and following \nany interruption of operations, during which time, contamination may have occured  utilizing the \nfollowing procedure: \n1. Wash- hot soapy water.\n2. Rinse-clear water.',
 '3. Sanitize-approved sanitizer and water.\n4. Air-dry.',
 'Non Critical Comments',
 '(330)(cid:13)\n* Operating without a permit.',
 '6. Hot Holding Temperatures Satisfactory',
 '* Sushi rice pH 3.99.',
 'Non Critical Violations:',
 '(330)(cid:13)',
 '* Operating without a permit.',
 'Client #',
 '201102240001',
 'Page 2 of 2',
 '201103080007',
 '\x0c']

In [128]:
v = t[1].split('Other Assesment observations and comments:\n\n')[1].split('Client #\n\n')[0].split('\n')
v

['21',
 '',
 'Certified Food Protection Manager',
 '',
 'Certified manager not present.',
 ' 336',
 'Provide Certified Food Protection Manager to premises during all hours of operation',
 'Certified Food Protection Manager Course schedule left with person in charge',
 '',
 'Non Critical Comments',
 '',
 '(303) Unlabeled container in spice room.  Containers storing food must be labeled.(cid:13)',
 '(305) Bowls used as scoops in spice room. Use scoops with handles.(cid:13)',
 '      Handle of scoop in contact with ice at soda station.  Store so handle is not in contact with ice.',
 '(322) Boxes on top shelf in walk in are blocking light.  Store boxes so light is not blocked.',
 '',
 'Non Critical Violations:',
 '(303) Unlabeled container in spice room.  Containers storing food must be labeled.(cid:13)',
 '',
 '(305) Bowls used as scoops in spice room. Use scoops with handles.(cid:13)',
 '',
 '      Handle of scoop in contact with ice at soda station.  Store so handle is not in contact wi

In [36]:
[x.split(' 3') for x in v]

[['7'],
 ['Cold Holding Temperatures'],
 ['*HIGH RISK***(cid:13)',
  'Federal Grab-n-Go cooler ambient air temp at 45F.  Thermometer in unit (at back, near cooling ',
  'elements) reading 32F.  Additional thermometer placed at front of middle shelf of unit.  Items inside',
  'unit at 45-46F.  Covers placed over unit, unit turned down further.  If unable to reach 41F or below, ',
  'relocate items to walk-in cooler.  Items for display in grab-n-go must be labeled as such.  ',
  ' 305',
  'Food must be held at 41\xc2\xb0 F or below',
  'Discard or remove food from sale'],
 ['8'],
 ['Facilities to Maintain Temperature'],
 [' 305',
  'Provide enough conveniently located, approved, properly operating refrigeration units to assure the ',
  'maintenance and cooling of foods at required temperatures.'],
 ['24'],
 ['Fabrication, Design, Installation and Maintenance'],
 ['*LOW RISK*(cid:13)',
  'Drain tray of soda fountain is cracked, accumulating residue.  Facility stated they have contacted ',

In [33]:
[x.replace('\n','').split('(cid:13)') for x in v]

[['7'],
 ['Cold Holding Temperatures'],
 ['*HIGH RISK***',
  'Federal Grab-n-Go cooler ambient air temp at 45F.  Thermometer in unit (at back, near cooling elements) reading 32F.  Additional thermometer placed at front of middle shelf of unit.  Items insideunit at 45-46F.  Covers placed over unit, unit turned down further.  If unable to reach 41F or below, relocate items to walk-in cooler.  Items for display in grab-n-go must be labeled as such.   305Food must be held at 41\xc2\xb0 F or belowDiscard or remove food from sale'],
 ['8'],
 ['Facilities to Maintain Temperature'],
 [' 305Provide enough conveniently located, approved, properly operating refrigeration units to assure the maintenance and cooling of foods at required temperatures.'],
 ['24'],
 ['Fabrication, Design, Installation and Maintenance'],
 ['*LOW RISK*',
  'Drain tray of soda fountain is cracked, accumulating residue.  Facility stated they have contacted Pepsi company for service. 309Repair / replace equipment'],
 ['27'

In [84]:
z = requests.get('http://appsrv.achd.net/reports/rwservlet?food_rep_insp&P_ENCOUNTER=201401010018')
soup = BeautifulSoup(z.content, from_encoding='UTF-8')
test = soup.findAll('body')[0]

In [71]:
z2 = requests.get('http://appsrv.achd.net/reports/rwservlet?food_rep_insp&P_ENCOUNTER=201409080018')
soup2 = BeautifulSoup(z2.content, from_encoding='UTF-8')
soup2
test2 = soup2.findAll('body')[0]
test2

<body><p>%PDF-1.4
1 0 obj
&gt;
endobj
5 0 obj
&gt;
stream
Gat=0?$"c1'n+D%rW?g,U$bZd'.!@H]X`3kff#=BD)XOnV[p':g[%P,QDB_S^V0\f6!f[5Un.%7
S`j0orVDe8dUW75/4ko/1q5`!rT`Hl(^Oh&gt;]Y8aklKE&gt;r
;&gt;1T9pCs-_O1XO'pn1ZWjrl]cqc20_eZS^hE;e\GY9L56NmhAZo!YK?FS*9uf&gt;6#&amp;:?bZSFfjr_
Qj;2+998[LRjYAqUJ&amp;!J/+WC/;u7f\L%/`'m(_PGCRXG,1(S93_+CmH_0HprrUm5\=INi!WtH!`
b=M8b&amp;qL&gt;#7Ra#4iA/Wp:uo(%KoDe8ZAs&gt;IL2j^;R<c>A^[&gt;'M4Eb1_.oMPd@k*io0n2o3II+He.9"4JM&gt;dt_tAh?HkGM^6I^Gpm7n
IeA\K,*jB&amp;)bX7*g[@9]hK]o;H!d\^[[9UKj%'B4/;h&amp;oG/59=q-nKMW)6&amp;!`1s!<f>10B
9t+%KLRB<oaozdhr c="" c0="">@kBU&gt;o[m\
ic.&amp;PVlPW?AhI1?.*LM+h0j7E[X6j.k-7hWaJW(7=kZ_+M4.]J\5cOJ])GUWAb@m#)!:E8oe^Y6
0@b^'4YfB'0'+WY\]br9EasKc\;ksu@[<_3pe>Q1]n^d]4/8RNiY%QfP]9dX!3\t
eFcd\eDIHIMiAcNHK$hon;En0NLBOBMGXgOM&gt;Le%%=c_`7]eA^%B&amp;lnR8@/6R/sAE/fK=%iFMgUtFKRUH9rAeu
-Kr6n?0Of/BM:A,7UF!nGaZd#%@94&gt;<b>Abo2*(srPK2+R7MV`jCK6VmCXouLlBS3Fm#9F4o!_p#FedO^UTkjdo.*k+
*3NmDL54B%*q-r2LqNoLnDqJgNWmF
3M/0dYb-^63[q"ekZ#-#D.&gt;er\%c^4Cd/b_cfZ

In [98]:
len(test2.findAll('p')[0])

2

In [97]:
len(test.findAll('p')[0])

1

In [65]:
test.get_text()

u'\nOracle Reports Services - Servlet\n\n\n \r\nfunction autoclose () \r\n{ \r\n  var curloc = window.location.href; \r\n  if (curloc.toLowerCase ().indexOf("closewindow=yes") != -1) \r\n  { \r\n    var imgs = document.getElementsByTagName(\'img\'); \r\n    for (var i=0;i<imgs.length;i++) \r\n    { \r\n      // Don\'t close the window if the execution is not successful\r\n      if (imgs[i].src.indexOf("erroricon.gif")!= -1) \r\n      { \r\n        return; \r\n      } \r\n    } \r\n    // Try to close the window . Works in IE6 / IE7 / Firefox 1.5 \r\n    // Does not work in Firefox 2.0 \r\n    window.opener=\'X\';\r\n    window.open(\'\',\'_parent\',\'\');\r\n    window.close();\r\n\r\n  }\r\n}\r\n\n'

In [104]:
len(soup2)

1

In [105]:
len(soup)

2

In [109]:
z.headers['Content-Type']

'text/html; charset=ISO-8859-1'

In [108]:
z2.headers['Content-Type']

'application/pdf'

# Scrape Data:

### Initial scrape:

In [151]:
start = pd.datetime(2011,1,1)
delta = pd.Timedelta(1, 'd')

In [137]:
pdfs = {}
for i in xrange(10):
    base = (start + i*delta).strftime("%Y%m%d")
    for j in xrange(100):
        link = 'http://appsrv.achd.net/reports/rwservlet?food_rep_insp&P_ENCOUNTER=%s%04d' % (base,j+1)
        z = requests.get(link)
        if z.headers['Content-Type'] == 'application/pdf':
            pdfs['%s%04d' % (base, j+1)] = {'link': link, 'text': convert_pdf_to_txt(link)}
        else:
            print '%s max: %d' % (base, j)
            break

20110101 max: 0
20110102 max: 0
20110103 max: 32
20110104 max: 33
20110105 max: 28
20110106 max: 53
20110107 max: 41
20110108 max: 0
20110109 max: 0
20110110 max: 38


In [114]:
start.strftime("%Y%m%d")

'20110101'

In [15]:
pdf_main = {}

In [16]:
pdf_main = merge_two_dicts(pdf_main, pdfs)

### Try more days:

In [20]:
start = pd.datetime(2011,5,5)
delta = pd.Timedelta(1, 'd')
step_size = 7
n_loops = 12

for i in xrange(n_loops):
    print '[%02d] START: %s' % (i, start.strftime("%Y%m%d"))
    pdfs = get_pdf_text(start, step_size)
    save_pdf_text(pdfs, 'data/pitt/pitt_%s.pkl' % start.strftime("%Y%m%d"))
    pdf_main = merge_two_dicts(pdf_main, pdfs)
    start += step_size * delta
    

[00] START: 20110505
20110505 max: 52
20110506 max: 55
20110507 max: 0
20110508 max: 0
20110509 max: 46
20110510 max: 39
20110511 max: 40
[01] START: 20110512
20110512 max: 40
20110513 max: 50
20110514 max: 1
20110515 max: 0
20110516 max: 19
20110517 max: 0
20110518 max: 37
[02] START: 20110519
20110519 max: 49
20110520 max: 46
20110521 max: 0
20110522 max: 0
20110523 max: 45
20110524 max: 41
20110525 max: 67
[03] START: 20110526
20110526 max: 86
20110527 max: 42
20110528 max: 0
20110529 max: 0
20110530 max: 0
20110531 max: 40
20110601 max: 61
[04] START: 20110602
20110602 max: 32
20110603 max: 29
20110604 max: 0
20110605 max: 0
20110606 max: 55
20110607 max: 38
20110608 max: 71
[05] START: 20110609
20110609 max: 56
20110610 max: 36
20110611 max: 0
20110612 max: 0
20110613 max: 56


KeyboardInterrupt: 

In [148]:
pdf_main_copy = pdf_main.copy()

In [205]:
test = pd.DataFrame.from_records(pdf_main).T

test2 = pd.DataFrame({'tag':test.index, 'date':test.index},dtype=int)
test2['date'] = (test2.date/10000).astype(int)

test2.info()
test2.groupby('date').count()

In [21]:
200000./60/60/24

2.314814814814815

# Read in files:

In [179]:
def read_in_dicts(f_list):
    f_path = 'data/pitt/pitt'
    d = {}
    for f in f_list:
        d = merge_two_dicts(d, pickle.load(open('%s_%s.pkl' % (f_path,f))))
    return d



In [178]:
f_list = ['20110101','20110111','20110112','20110214','20110221','20110228','20110302',
          '20110303','20110310','20110317','20110324','20110331','20110407','20110414',
          '20110421','20110428','20110505','20110512','20110519','20110526']
f_list = ['20110602','20110609','20110616','20110623','20110630','20110707','20110714',
          '20110721','20110728','20110804','20110811','20110818','20110825','20110901',
          '20110908','20110915','20110922','20110929','20111006','20111013','20111020',
          '20111027','20111103','20111110','20111117','20111124','20111201','20111208',
          '20111215','20111222','20111229']
#f_list = ['20120105','20120112','20120119','2012026']
f_path = 'data/pitt/pitt'
d = {}
for f in f_list:
    d = merge_two_dicts(d, pickle.load(open('%s_%s.pkl' % (f_path,f))))

save_pdf_text(d, '%s_%s_to_%s.pkl' % (f_path,f_list[0],f))

In [15]:
f_list = pd.read_csv('../data/pitt/ls_pitt_parts.csv', header=None).values.flatten().tolist()
f_path = '../data/pitt/'
d = {}
for f in f_list:
    with open(f_path+f) as fname:
        d = merge_two_dicts(d, pickle.load(fname))

save_pdf_text(d, '%spitt_20110101_to_20151015.pkl' % (f_path))

In [16]:
inspec = pd.DataFrame.from_records(d).T

df = pd.DataFrame({'tag':inspec.index, 'date':inspec.index},dtype=int)
df['date'] = (df.date/10000).astype(int)

df.info()

df.groupby('date').count()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52562 entries, 0 to 52561
Data columns (total 2 columns):
date    52562 non-null int64
tag     52562 non-null int64
dtypes: int64(2)
memory usage: 1.2 MB


,tag
date,
20110103,32
20110104,33
20110105,28
20110106,53
20110107,41
20110110,38
20110111,44
20110112,45
20110113,50


In [25]:
inspec.head()

,link,text
201101030001,http://appsrv.achd.net/reports/rwservlet?food_...,Allegheny County Health Department\nFood Safet...
201101030002,http://appsrv.achd.net/reports/rwservlet?food_...,Allegheny County Health Department\nFood Safet...
201101030003,http://appsrv.achd.net/reports/rwservlet?food_...,Allegheny County Health Department\nFood Safet...
201101030004,http://appsrv.achd.net/reports/rwservlet?food_...,Allegheny County Health Department\nFood Safet...
201101030005,http://appsrv.achd.net/reports/rwservlet?food_...,Allegheny County Health Department\nFood Safet...
